# Лабораторная работа #2

## Сбор данных для обучения

Датасет был собран путём парсинга сайта https://www.anekdot.ru/. Взял все анекдоты за 2021 год и 2020 года, всего около 54,000 штук.
Для парсинга использовал библиотку `BeautifulSoup4`

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from itertools import product

years = ['2020', '2021']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11']
days = ['01', '02', '03', '04', '05', '06', '07', '08', '09'] + [str(x) for x in range(10, 29)]
base_url = 'https://www.anekdot.ru/release/anekdot/day/'

jokes = []
for (year, month, day) in product(years, months, days):
    if day == '01':
        print(f'{year}.{month}...')

    url = base_url + year + '-' + month + '-' + day + '/'
    page = requests.get(url)

    soup = BeautifulSoup(page.content, "html.parser")
    search_result = soup.find_all('div', class_='topicbox')

    for box in search_result:
        text = box.findAll('div', class_='text')
        if len(text):
            jokes.append(text[0].get_text().replace('\r', ' ').replace('\xad', ' '))

print(f'Number of jokes: {len(jokes)}')

df = pd.DataFrame({'Jokes': jokes})

## Построение модели

In [1]:
import numpy as np
import pandas as pd

Читаем полученный ранее датасет и оставляем первые 9000 анекдотов, чтобы данные влезали в память.

Уверен, есть способ обучиться на всём датасете, но я его не нашёл :(

In [2]:
jokes_df = pd.read_csv('jokes.csv')
jokes = jokes_df.Jokes.values[:9000]

Создаём токенайзер, который будет переводить слова в числа

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 50000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(jokes)
sequences = tokenizer.texts_to_sequences(jokes)
vocab_size = len(tokenizer.word_index)

Обучающая выборка представляет из себя наборы из 6 слов. По певым пяти пытаемся предсказать следующее слово из анекдота

In [6]:
sentence_len = 6
pred_len = 1
train_len = sentence_len - pred_len
seq = []

for joke in sequences:
    if len(joke) >= sentence_len:
        for i in range(len(joke) - sentence_len):
            seq.append(joke[i: i + sentence_len])

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

x_train = np.array([i[:train_len] for i in seq])
y_train = np.array([i[-1] for i in seq])

In [7]:
x_train.shape, y_train.shape, vocab_size

((191762, 5), (191762,), 40173)

Пердставляем `y_train[i]` в виде вектора, в котором есть одна единица, которая соответсвует слову из вокабуляра, остальные все нули

In [8]:
dummies = pd.get_dummies(y_train, columns=[str(i) for i in range(1, vocab_size + 1)])

new_dummies = {}
for i in range(1, vocab_size + 1):
    if i in dummies.columns:
        new_dummies.update({i: dummies[i]})
    else:
        new_dummies.update({i: np.zeros(y_train.shape[0])})
new_dummies = pd.DataFrame(new_dummies)

In [9]:
new_dummies.shape

(191762, 40173)

Таким образом, задача преобразуется в задачу классификации, где по пяти словам надо предсказать одно слово из вокабуляра.

Создаём сетку для решения этой задачи

In [1]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(vocab_size + 1, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(vocab_size, activation='softmax')
])

KeyboardInterrupt: 

Использум `ModelCheckpoint`, чтобы сохранять прогресс

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
filepath = "./models/model_weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Обучаем сетку на 100 эпохах

In [12]:
history = model.fit(x_train, new_dummies, epochs=100,
                    batch_size=128, callbacks=callbacks_list, verbose=1)

Epoch 1/100
1498/1499 [============================>.] - ETA: 0s - loss: 8.6690 - accuracy: 0.0310
Epoch 00001: loss improved from inf to 8.66899, saving model to ./models\model_weights.hdf5
1499/1499 [==============================] - 157s 103ms/step - loss: 8.6690 - accuracy: 0.0310
Epoch 2/100
1498/1499 [============================>.] - ETA: 0s - loss: 8.2822 - accuracy: 0.0319
Epoch 00002: loss improved from 8.66899 to 8.28243, saving model to ./models\model_weights.hdf5
1499/1499 [==============================] - 158s 106ms/step - loss: 8.2824 - accuracy: 0.0319
Epoch 3/100
1498/1499 [============================>.] - ETA: 0s - loss: 7.9404 - accuracy: 0.0368
Epoch 00003: loss improved from 8.28243 to 7.94039, saving model to ./models\model_weights.hdf5
1499/1499 [==============================] - 160s 107ms/step - loss: 7.9404 - accuracy: 0.0368
Epoch 4/100
1498/1499 [============================>.] - ETA: 0s - loss: 7.6049 - accuracy: 0.0434
Epoch 00004: loss improved from 7.9

На предикшене задаём начало анекдота, берём на каждой итерации последние 5 слов и пытаемся предсказать следующее

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate(model, seq, max_len=20):
    tokenized_sent = tokenizer.texts_to_sequences([seq])
    max_len = max_len + len(tokenized_sent[0])

    while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-5:], maxlen=5)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax() + 1)

    return seq + ' ' + " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))

Вот пара примеров работы

In [14]:
generate(model, 'Ехали в поезде')

'Ехали в поезде в поезде коллегами сотрудников отменить его отдых из местных малого части к ситуации к том что в самоизоляции кричал робинзон крузо во'

In [15]:
generate(model, 'Приятель из Питера Сергей Дединский')

'Приятель из Питера Сергей Дединский приятель из питера сергей одиночные выходные при масках маникюр на нашей конституции просто подвергнут появился эксперты к царе менеджеров году телевизор© люди что российская'

Получилось не особо смешно конечно. Думаю, основная причина - это то, что из-за того что не получилось разобраться, как обойти ограничение по памяти, обучающая выборка оказалась слишком маленькой, и сетка не смогла нормально натренироваться, чтобы предсказывать что-то адекватное. Возможно, ещё стоило поэксперементировать с разными архитектурами, и поподбирать оптимальное количество слов, по которым предсказывается следующее